In [1]:
import numpy as np
from rlbench.action_modes.action_mode import MoveArmThenGripper
from rlbench.action_modes.arm_action_modes import ArmActionMode, JointVelocity, JointPosition, EndEffectorPoseViaPlanning, EndEffectorPoseViaIK


from rlbench.action_modes.gripper_action_modes import Discrete
from rlbench.environment import Environment
from rlbench.observation_config import ObservationConfig, CameraConfig
# from rlbench.tasks.pick_described_object import PickDescribedObject
from rlbench.tasks import PutGroceriesInCupboard, PickAndLift, StackBlocks, PlaceHangerOnRack, PickDescribedObject, TakeLidOffSaucepan, SetTheTable
from scipy.spatial.transform import Rotation as R
from matplotlib import pyplot as plt
from rlbench.sim2real.domain_randomization import RandomizationConfig


class Agent(object):
    def __init__(self, action_shape):
        self.action_shape = action_shape

    def act(self, obs):
        xyz = np.array([2.78478146e-01, -8.15162994e-03,  1.47194254e+00])
        rot = np.array([-3.14, 0.24243259,  3.14])
        #rot to quat
        quat = R.from_euler('xyz', rot).as_quat()
        gripper = [1.0]
        return np.concatenate([xyz,quat, gripper], axis=-1)


In [2]:
camera = CameraConfig(image_size=(224, 224), depth=False, point_cloud=False, mask=False)
obs_config = ObservationConfig(left_shoulder_camera=camera, right_shoulder_camera=camera, front_camera=camera, overhead_camera=camera)

env = Environment(
    action_mode=MoveArmThenGripper(
        arm_action_mode=EndEffectorPoseViaPlanning(absolute_mode=True, collision_checking=True), gripper_action_mode=Discrete()),
    obs_config=obs_config,
    robot_setup='panda',
    # randomize_every=1,
    # visual_randomization_config=RandomizationConfig(),
    # dynamics_randomization_config=RandomizationConfig(),
    headless=False)
env.launch()



In [3]:
target_item_poses = []
waypoints = []
gripper_poses = []
front_rgbs = []
def callable_fun(obs, task, variation_num):
    # target item pose
    target_item_pose = task._task.get_graspable_objects()[variation_num].get_pose()
    target_item_pose = np.concatenate([target_item_pose[:3], R.from_quat(target_item_pose[3:]).as_euler('xyz')])
    target_item_poses.append(target_item_pose)

    #waypoints
    current_pose = np.array(obs.gripper_pose)
    current_pose = np.concatenate([current_pose[:3], R.from_quat(current_pose[3:]).as_euler('xyz')])
    gripper_poses.append(current_pose)
    wps = [wp._waypoint.get_position() for wp in task._task._waypoints]
    
    if abs(current_pose[:3] - wps[0]).mean() < 1e-2:
        waypoints.append(0)
    elif abs(current_pose[:3] - wps[1]).mean() < 1e-2:
        waypoints.append(1)
    elif abs(current_pose[:3] - wps[2]).mean() < 1e-2:
        waypoints.append(2)
    else:
        waypoints.append(-1)

    #front rgb
    front_rgbs.append(obs.front_rgb)
    

variation_num = 1
task = env.get_task(PickDescribedObject)
task.set_variation(variation_num)


# demos = task.get_demos(1, live_demos=True, callable_each_step=lambda x: callable_fun(x,task=task, variation_num=variation_num)) 
# basket_position = task._task.dropin_box.get_position()
# demos = np.array(demos).flatten()
# descriptions, obs = task.reset() 

In [11]:
lower_bound = np.array([task._scene._workspace_minx, task._scene._workspace_miny, task._scene._workspace_minz])
upper_bound = np.array([task._scene._workspace_maxx, task._scene._workspace_maxy, task._scene._workspace_maxz])

In [12]:
lower_bound

array([-0.27499999, -0.65500004,  0.75199986])

In [13]:
upper_bound

array([0.77499999, 0.65500004, 1.75199986])

In [5]:
task.reset()


(['put the soup in the basket',
  'pick up the soup and place in the basket',
  'I want to put away the soup'],
 <rlbench.backend.observation.Observation at 0x77888e2ec550>)

In [6]:
np.random.uniform(-3.14, 3.14, 1)

array([-0.09317659])

In [7]:
np.concatenate([[0,0],np.random.uniform(-3.14, 3.14, 1)])

array([ 0.        ,  0.        , -2.47777621])

In [9]:
while True:
    try:
        task.reset()
        [g.set_mass(10) for g in task._scene.task.get_graspable_objects()]
        pos = np.random.uniform(lower_bound, upper_bound)
        rot = np.random.uniform(-3.14, 3.14, 3)
        joint_position = task._scene.robot.arm.solve_ik_via_sampling(position=pos, euler=rot)
        print(joint_position)
        task._scene.robot.arm.set_joint_positions(joint_position[0], disable_dynamics=True)
        task._scene.get_demo() 
    except Exception as e:
        print(e)
        continue

[[-1.91228473  1.24422979  2.64241219 -2.86185598 -2.24492693  1.0006876
  -0.9034161 ]]
Error in task pick_described_object. Demo was completed, but was not successful.
Could not find a valid joint configuration for desired end effector pose.
[[ 0.37096265  0.69926065  0.8678354  -1.48930252  0.01049488  1.96434546
   2.09725189]]
Could not find a valid joint configuration for desired end effector pose.
Could not find a valid joint configuration for desired end effector pose.


KeyboardInterrupt: 

In [ ]:
pos

In [ ]:
t[0]

In [ ]:
task._task.robot.arm.get_joint_positions()

In [ ]:
task._task.robot.arm.set_joint_positions(t, disable_dynamics=False)

In [ ]:
pose = task._scene.robot.arm.get_pose()

In [ ]:
pose = task._scene.robot.gripper.get_pose()

In [ ]:
pose

In [ ]:
task._scene.robot.arm.get_configs_for_tip_pose(position=pose[:3], quaternion= pose[3:])

In [ ]:
task._scene.robot.arm.get_configs_for_tip_pose(position=pos, euler = rot)

In [ ]:
task._scene.robot.gripper.joints

In [ ]:
waypoints.index(2)

In [ ]:
task._task._waypoints[0]._waypoint.get_pose()

In [ ]:
task._task.grasp_points[0].get_pose()

In [ ]:
task._task.get_graspable_objects()[0].get_pose()

In [ ]:
task._task.robot.gripper

In [ ]:
task._task._waypoints[0].get_waypoint_object().get_pose()
task.step(np.concatenate([task._task._waypoints[0].get_waypoint_object().get_pose(),[1]]))

In [ ]:
task.step(np.concatenate([task._task.target_block.get_pose(),[0]]))

In [ ]:
task.step(np.concatenate([task._task.target_block.get_pose(),[0]]))

In [ ]:
# task._task.distractors[0].get_pose()
task.step(np.concatenate([task._task.distractors[0].get_pose(),[0]]))

In [ ]:
task = env.get_task(PickAndLift)

agent = Agent(env.action_shape)
training_steps = 120
episode_length = 40
obs = None
for i in range(training_steps):
    # if i % episode_length == 0:
    print("Reset Episode")
    descriptions, obs = task.reset()
    print(descriptions)
    item_name = task._task.item_name
    item_coord = task._task.item.get_position()
    item_orientation = task._task.item.get_orientation()
    print(item_name)
    print(item_coord)
    print(item_orientation)
    action = agent.act(obs)
    obs, reward, terminate = task.step(action)
    print(reward)
    break


In [ ]:
task._task.get_waypoints()[0].get_waypoint_object().get_position()

In [10]:

print("Done")
env.shutdown()

Done
[CoppeliaSim:loadinfo]   done.
